In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns #Control figure 
import numpy as np
import os
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
from sodapy import Socrata

In [3]:
#MyAppToken = ''

In [4]:
#client = Socrata("data.cityofnewyork.us", MyAppToken)

In [5]:
#results = client.get("dsg6-ifza", limit=2000)

In [6]:
#df = pd.DataFrame.from_records(results)

In [7]:
cwd = os.getcwd() #to get current working directory
#print(cwd)

In [8]:
df = pd.read_csv('DOHMH_Childcare_Center_Inspections.csv', encoding = "iso-8859-1")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53200 entries, 0 to 53199
Data columns (total 34 columns):
Center Name                                    53200 non-null object
Legal Name                                     53200 non-null object
Building                                       53192 non-null object
Street                                         53200 non-null object
Borough                                        53200 non-null object
ZipCode                                        53123 non-null float64
Phone                                          53126 non-null object
Permit Number                                  47043 non-null float64
Permit Expiration                              53200 non-null object
Status                                         53200 non-null object
Age Range                                      50906 non-null object
Maximum Capacity                               53200 non-null int64
Day Care ID                                    53200 non-null 

In [10]:
def clean_string(astr):
    return astr.lower().replace('.', '') \
        .replace(',', '') \
        .replace(';', '') \
        .replace(':', '') \
        .replace('á', 'a') \
        .replace('é', 'e') \
        .replace('í', 'i') \
        .replace('ó', 'o') \
        .replace('ú', 'u') \
        .replace(' ', '_') \
        .replace('ñ', 'ni')

In [11]:
def clean_columns(df):
    for series in df:
        df.rename(columns={series:clean_string(series)}, inplace=True)

In [12]:
def execute(raw_dataset_path, clean_dataset_path):
    print("\t-> Leyendo datos crudos a un DataFrame")
    df = pd.read_csv('DOHMH_Childcare_Center_Inspections.csv')

In [13]:
print("\t-> Limpando columnas")

	-> Limpando columnas


In [14]:
clean_columns(df)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53200 entries, 0 to 53199
Data columns (total 34 columns):
center_name                                    53200 non-null object
legal_name                                     53200 non-null object
building                                       53192 non-null object
street                                         53200 non-null object
borough                                        53200 non-null object
zipcode                                        53123 non-null float64
phone                                          53126 non-null object
permit_number                                  47043 non-null float64
permit_expiration                              53200 non-null object
status                                         53200 non-null object
age_range                                      50906 non-null object
maximum_capacity                               53200 non-null int64
day_care_id                                    53200 non-null 

In [16]:
df.isnull().sum()

center_name                                        0
legal_name                                         0
building                                           8
street                                             0
borough                                            0
zipcode                                           77
phone                                             74
permit_number                                   6157
permit_expiration                                  0
status                                             0
age_range                                       2294
maximum_capacity                                   0
day_care_id                                        0
program_type                                       0
facility_type                                      0
child_care_type                                    0
building_identification_number                     0
url                                            27067
date_permitted                                

In [17]:
print("\t-> Reemplazando espacios en blanco")
for col in df.select_dtypes('object'):
    df[col] = df[col].replace('\s+', ' ', regex=True)

	-> Reemplazando espacios en blanco


In [18]:
print("\t-> Limpiando valores")
for col in df.select_dtypes('object'):
        df[col] = df[col].str.strip()
        df[col] = df[col].str.lower()
        df[col] = df[col].str.replace('á', 'a')
        df[col] = df[col].str.replace('é', 'e')
        df[col] = df[col].str.replace('í', 'i')
        df[col] = df[col].str.replace('ó', 'o')
        df[col] = df[col].str.replace('ú', 'u')
        df[col] = df[col].str.replace(' ', '_')

	-> Limpiando valores


In [19]:
print("\t-> Cambiando NA por np.nan")
for col in df.select_dtypes('object'):
    df.loc[df[col] == 'na', col] = np.nan

	-> Cambiando NA por np.nan


In [20]:
df.isnull().sum()

center_name                                        0
legal_name                                         0
building                                           8
street                                             0
borough                                            0
zipcode                                           77
phone                                             74
permit_number                                   6157
permit_expiration                                  0
status                                             0
age_range                                       2294
maximum_capacity                                   0
day_care_id                                        0
program_type                                       0
facility_type                                      0
child_care_type                                    0
building_identification_number                     0
url                                            27067
date_permitted                                

In [21]:
df.to_csv('df.csv', index=False)

In [22]:
print("\t-> Cuántos valores NaN tiene la base")

	-> Cuántos valores NaN tiene la base


In [23]:
df.isnull().sum()

center_name                                        0
legal_name                                         0
building                                           8
street                                             0
borough                                            0
zipcode                                           77
phone                                             74
permit_number                                   6157
permit_expiration                                  0
status                                             0
age_range                                       2294
maximum_capacity                                   0
day_care_id                                        0
program_type                                       0
facility_type                                      0
child_care_type                                    0
building_identification_number                     0
url                                            27067
date_permitted                                

In [24]:
print("\t-> Eliminar duplicados")

	-> Eliminar duplicados


In [25]:
df.duplicated().sum()

13419

In [26]:
df = df.drop_duplicates()
df.shape

(39781, 34)

In [27]:
print("\t-> Reagrupar en dos variables Inspection Summary Result")

	-> Reagrupar en dos variables Inspection Summary Result


In [28]:
df.inspection_summary_result.value_counts(dropna=False) #Observar que existen 2,125 NaN

initial_annual_inspection_-_reinspection_required;_fines_pending                                                10046
initial_annual_inspection_-_passed_inspection_with_no_violations                                                 6715
initial_annual_inspection_-_reinspection_required                                                                4302
initial_annual_inspection_-_reinspection_not_required                                                            3628
monitoring_inspection_non-routine_-_passed_inspection_with_no_violations                                         2275
compliance_inspection_of_open_violations_-_reinspection_required;_fines_pending                                  2139
NaN                                                                                                              2125
monitoring_inspection_non-routine_-_reinspection_required;_fines_pending                                         1724
compliance_inspection_of_open_violations_-_previously_ci

In [29]:
df_1 = df.inspection_summary_result.str.partition('_-_')

In [30]:
df_1

,0,1,2
0,initial_annual_inspection,_-_,reinspection_required
1,compliance_inspection_of_open_violations,_-_,previously_cited_violations_corrected
2,initial_annual_inspection,_-_,reinspection_required
3,compliance_inspection_of_open_violations,_-_,previously_cited_violations_corrected
4,initial_annual_inspection,_-_,reinspection_required
...,...,...,...
53195,compliance_inspection_of_open_violations,_-_,reinspection_required;_fines_pending
53196,initial_annual_inspection,_-_,reinspection_required;_fines_pending
53197,initial_annual_inspection,_-_,reinspection_required;_fines_pending
53198,initial_annual_inspection,_-_,passed_inspection_with_no_violations


In [31]:
df_1.rename(columns={0:'inspection_reason', 2:'result'}, inplace=True)

In [32]:
df_1.inspection_reason.value_counts(dropna=False)

initial_annual_inspection                       25233
compliance_inspection_of_open_violations         7022
monitoring_inspection_non-routine                5323
NaN                                              2125
lead_based_paint_or_lead_in_water_inspection       78
Name: inspection_reason, dtype: int64

In [33]:
df_1.result.value_counts(dropna=False)

reinspection_required;_fines_pending                                 13917
passed_inspection_with_no_violations                                  9236
reinspection_required                                                 5543
reinspection_not_required                                             4657
NaN                                                                   2125
previously_cited_violations_corrected                                 1876
reinspection_required;_violations_corrected_at_time_of_inspection     1519
passed_inspection;_violations_corrected_at_time_of_inspection          573
reinspection_required;_program_closed                                  248
previously_closed_program_re-opened                                     87
Name: result, dtype: int64

In [34]:
df_1

,inspection_reason,1,result
0,initial_annual_inspection,_-_,reinspection_required
1,compliance_inspection_of_open_violations,_-_,previously_cited_violations_corrected
2,initial_annual_inspection,_-_,reinspection_required
3,compliance_inspection_of_open_violations,_-_,previously_cited_violations_corrected
4,initial_annual_inspection,_-_,reinspection_required
...,...,...,...
53195,compliance_inspection_of_open_violations,_-_,reinspection_required;_fines_pending
53196,initial_annual_inspection,_-_,reinspection_required;_fines_pending
53197,initial_annual_inspection,_-_,reinspection_required;_fines_pending
53198,initial_annual_inspection,_-_,passed_inspection_with_no_violations


In [35]:
df_1 = df_1.drop(df_1.columns[[1]], axis=1) 

In [36]:
df_1

,inspection_reason,result
0,initial_annual_inspection,reinspection_required
1,compliance_inspection_of_open_violations,previously_cited_violations_corrected
2,initial_annual_inspection,reinspection_required
3,compliance_inspection_of_open_violations,previously_cited_violations_corrected
4,initial_annual_inspection,reinspection_required
...,...,...
53195,compliance_inspection_of_open_violations,reinspection_required;_fines_pending
53196,initial_annual_inspection,reinspection_required;_fines_pending
53197,initial_annual_inspection,reinspection_required;_fines_pending
53198,initial_annual_inspection,passed_inspection_with_no_violations


In [37]:
df = pd.concat([df, df_1], axis=1)

In [38]:
df.shape

(39781, 36)

In [39]:
df.head()

,center_name,legal_name,building,street,borough,zipcode,phone,permit_number,permit_expiration,status,...,critical_violation_rate,average_critical_violation_rate,inspection_date,regulation_summary,violation_category,health_code_sub_section,violation_status,inspection_summary_result,inspection_reason,result
0,yeshivat_ohr_haiim,yeshivat_ohr_haiim,86-06,135th_street,queens,11418.0,718-658-7066,NaN,01/23/2115,active,...,100.0,NaN,01/31/2019,written_safety_plan_describes_provisions_for_h...,general,43.07(b)(4),corrected,initial_annual_inspection_-_reinspection_required,initial_annual_inspection,reinspection_required
1,yeshivat_ohr_haiim,yeshivat_ohr_haiim,86-06,135th_street,queens,11418.0,718-658-7066,NaN,01/23/2115,active,...,100.0,NaN,05/30/2018,there_were_no_new_violations_observed_at_the_t...,NaN,NaN,NaN,compliance_inspection_of_open_violations_-_pre...,compliance_inspection_of_open_violations,previously_cited_violations_corrected
2,yeshivat_ohr_haiim,yeshivat_ohr_haiim,86-06,135th_street,queens,11418.0,718-658-7066,NaN,01/23/2115,active,...,100.0,NaN,01/31/2019,children_enrolled_in_school-based_child_care_p...,critical,43.17(a)(1),corrected,initial_annual_inspection_-_reinspection_required,initial_annual_inspection,reinspection_required
3,yeshivat_ohr_haiim,yeshivat_ohr_haiim,86-06,135th_street,queens,11418.0,718-658-7066,NaN,01/23/2115,active,...,100.0,NaN,04/16/2019,there_were_no_new_violations_observed_at_the_t...,NaN,NaN,NaN,compliance_inspection_of_open_violations_-_pre...,compliance_inspection_of_open_violations,previously_cited_violations_corrected
4,yeshivat_ohr_haiim,yeshivat_ohr_haiim,86-06,135th_street,queens,11418.0,718-658-7066,NaN,01/23/2115,active,...,100.0,NaN,04/17/2018,sufficient_handwash_sinks_with_hot_and_cold_ru...,critical,131.09(d)(2),corrected,initial_annual_inspection_-_reinspection_required,initial_annual_inspection,reinspection_required


In [40]:
print("\t-> Poner como primer columna center_id")

	-> Poner como primer columna center_id


In [41]:
df.rename(columns={'day_care_id':'center_id'}, inplace=True)

In [42]:
df.center_id

0        dc20398
1        dc20398
2        dc20398
3        dc20398
4        dc20398
          ...   
53195    dc23076
53196    dc11240
53197    dc22116
53198    dc22618
53199    dc25020
Name: center_id, Length: 39781, dtype: object

In [43]:
def order(frame,var): 
    varlist =[w for w in frame.columns if w not in var] 
    frame = frame[var+varlist] 
    return frame

In [44]:
df = order(df,['center_id'])

In [45]:
df.head()

,center_id,center_name,legal_name,building,street,borough,zipcode,phone,permit_number,permit_expiration,...,critical_violation_rate,average_critical_violation_rate,inspection_date,regulation_summary,violation_category,health_code_sub_section,violation_status,inspection_summary_result,inspection_reason,result
0,dc20398,yeshivat_ohr_haiim,yeshivat_ohr_haiim,86-06,135th_street,queens,11418.0,718-658-7066,NaN,01/23/2115,...,100.0,NaN,01/31/2019,written_safety_plan_describes_provisions_for_h...,general,43.07(b)(4),corrected,initial_annual_inspection_-_reinspection_required,initial_annual_inspection,reinspection_required
1,dc20398,yeshivat_ohr_haiim,yeshivat_ohr_haiim,86-06,135th_street,queens,11418.0,718-658-7066,NaN,01/23/2115,...,100.0,NaN,05/30/2018,there_were_no_new_violations_observed_at_the_t...,NaN,NaN,NaN,compliance_inspection_of_open_violations_-_pre...,compliance_inspection_of_open_violations,previously_cited_violations_corrected
2,dc20398,yeshivat_ohr_haiim,yeshivat_ohr_haiim,86-06,135th_street,queens,11418.0,718-658-7066,NaN,01/23/2115,...,100.0,NaN,01/31/2019,children_enrolled_in_school-based_child_care_p...,critical,43.17(a)(1),corrected,initial_annual_inspection_-_reinspection_required,initial_annual_inspection,reinspection_required
3,dc20398,yeshivat_ohr_haiim,yeshivat_ohr_haiim,86-06,135th_street,queens,11418.0,718-658-7066,NaN,01/23/2115,...,100.0,NaN,04/16/2019,there_were_no_new_violations_observed_at_the_t...,NaN,NaN,NaN,compliance_inspection_of_open_violations_-_pre...,compliance_inspection_of_open_violations,previously_cited_violations_corrected
4,dc20398,yeshivat_ohr_haiim,yeshivat_ohr_haiim,86-06,135th_street,queens,11418.0,718-658-7066,NaN,01/23/2115,...,100.0,NaN,04/17/2018,sufficient_handwash_sinks_with_hot_and_cold_ru...,critical,131.09(d)(2),corrected,initial_annual_inspection_-_reinspection_required,initial_annual_inspection,reinspection_required


In [46]:
df.to_csv('C:/Users/Elizabeth/pre_processed_3/Transform/df.csv', index=False)